In [2]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------------ --------------------- 5.2/11.1 MB 31.9 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 25.5 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 23.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------------- -------------------- 6.3/12.6 MB 29.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 30.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   ------ --------------------------------- 6.3/40.9 MB 38.6 MB/s eta 0:00:01
   ------- -------------------------------- 7.3/40.9 MB 16.8 MB/s eta 0:00:03
   ------------ --------------------------- 12.6/40.9 MB 22.5 MB/s eta 0:00:02
   ----------------- ---------------------- 17.6/40.9 MB 20.5 MB/s eta 0:00:02
   ---------------------- ----------------- 23.1/40.9 MB 23.2 MB/s eta 0:00:01
   -

# Hyper Parameter Tuning
- hyper parameter : 모델 설정과 관련해 직접 지정할 수 있는 매개변수
- model parameter : 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수

### GridSearchCV
- 내 모델에서 가장 잘 맞는 하이퍼파라미터 조합을 자동으로 찾아주는 도구 (from scikit-learn 라이브러리)


In [5]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 로드
iris_input, iris_target = load_iris(return_X_y=True)

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 값
params = {
    'n_neighbors': range(1, 13, 2)
}

# 첫 번째 인자: 모델
# 두 번째 인자: 테스트 할 파라미터 (딕셔너리)
# scoring: 평가 지표 (accuracy, precision, recall, f1)
# cv: 반복 횟수, ex) 5-fold cross validation -> 데이터를 5덩어리로 나누어서 4개는 학습, 1개는 검증, 이거를 5번 반복.
grid = GridSearchCV(knn, params, scoring='accuracy', cv=5) 
grid.fit(iris_input, iris_target)

print("최적의 파라미터:", grid.best_params_)
print("최적의 모델 객체:", grid.best_estimator_)
print("최적화된 점수:", grid.best_score_) # 각 반복에서 나온 점수의 평균을 grid.best_score_ 로 반환.

최적의 파라미터: {'n_neighbors': 7}
최적의 모델 객체: KNeighborsClassifier(n_neighbors=7)
최적화된 점수: 0.9800000000000001


In [6]:
best_knn = grid.best_estimator_
best_knn.fit(iris_input, iris_target)
best_knn.score(iris_input, iris_target)
# 최적의 모델 객체 n_neighbors': 7 로 학습 

0.9733333333333334

### RandomSearchCV
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다.
    - 탐색범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다.
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다.

##### 쉽게설명
- 하이퍼파라미터 조합을 전부 다 해보는 대신, 일부만 랜덤하게 골라서 실험하는 방식

🧩 다시 라면 예시! 🍜
이번엔 라면 끓이는 실험을 다시 해봅시다.

물 양: 400, 500, 600, 700, 800ml

끓이는 시간: 2분, 3분, 4분, 5분

계란: 있음, 없음

고추가루: 있음, 없음

모든 조합을 다 해보려면 5 × 4 × 2 × 2 = 80번 실험해야 해요.
→ 이게 GridSearchCV 방식이에요.

🍀 그런데 시간이 없다?
"그럼 10개만 랜덤하게 뽑아서 해보자!"

이게 바로 RandomizedSearchCV예요!
빠르게 대강 실험해보고 적당히 괜찮은 조합을 찾는 거죠.

In [7]:
from sklearn.model_selection import RandomizedSearchCV

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 생성
params = {
    'n_neighbors': range(1, 100, 2) # 1,3,5, ..., 99 
}

# n_iter: 탐색할 최적의 하이퍼 파라미터 조합 수 (기본값: 10) -> 후보 조합 중 10개를 랜덤하게 골라서 시도
#         값이 크면 시간이 오래 걸림 / 값이 작으면 좋은 조합을 찾을 가능성 저하 
# random_state=0: 랜덤 추출이지만 항상 동일한 결과를 원할 때 고정해주는 시드 값
rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0)
rd_search.fit(iris_input, iris_target)

print("최적의 파라미터:", rd_search.best_params_)
print("최적의 모델 객체:", rd_search.best_estimator_)
print("최적화된 점수:", rd_search.best_score_)
rd_search.cv_results_

c:\Users\hyuna\anaconda3\envs\pystudy_env\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\hyuna\anaconda3\envs\pystudy_env\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\hyuna\anaconda3\envs\pystudy_env\Lib\subprocess.py", line 550, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hyuna\anaconda3\envs\pystudy_env\Lib\subprocess.py", line 1028, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\hyuna\anaconda3\envs\pystudy_env\Lib\subprocess.py", line 154

최적의 파라미터: {'n_neighbors': 5}
최적의 모델 객체: KNeighborsClassifier()
최적화된 점수: 0.9733333333333334


{'mean_fit_time': array([0.0007823 , 0.00062656, 0.00061197, 0.00089788, 0.00050635,
        0.00080333, 0.00019574, 0.00039806, 0.00110941, 0.00040021]),
 'std_fit_time': array([0.00048243, 0.00059352, 0.00049972, 0.00090413, 0.00101271,
        0.0004017 , 0.00039148, 0.00048753, 0.00020987, 0.00049015]),
 'mean_score_time': array([0.00295558, 0.00157471, 0.00147929, 0.023313  , 0.00171595,
        0.00130739, 0.00820246, 0.00677481, 0.00100341, 0.00180302]),
 'std_score_time': array([1.93830351e-03, 4.95114912e-04, 4.76295130e-04, 1.29360710e-02,
        6.15590333e-04, 4.04482347e-04, 6.12701025e-04, 4.27007571e-04,
        7.42029711e-06, 5.12467114e-04]),
 'param_n_neighbors': masked_array(data=[57, 23, 21, 83, 5, 55, 77, 63, 45, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'n_neighbors': 57},
  {'n_neighbors': 23},
  {'n_neighbors': 21},
  {'n_neighbors': 83},
  {'n_nei

---

### HyperOpt
- 하이퍼파라미터를 "똑똑하게" 찾기 위한 자동 튜닝 라이브러리
- (Python 기반 오픈소스, 딥러닝/머신러닝 모두에서 사용 가능!)

📌 어디에 쓰일까?
모델 학습할 때 중요한 하이퍼파라미터들:

learning_rate

batch_size

n_estimators

dropout_rate

max_depth

...

이런 것들을 사람이 일일이 바꾸면서 실험하지 않고,
HyperOpt가 스스로 탐색하면서 최적값을 찾아주는 역할을 해요.

- 즉, 좋은 방향으로 똑똑하게 파라미터를 조정하고, 빠르고, 성능도 좋을 가능성이 높음

**hyper.hp클래스**
<table border="1">
  <thead>
    <tr>
      <th>함수명</th>
      <th>설명</th>
      <th>사용 방법</th>
      <th>예시 코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>hp.uniform</td>
      <td>연속적인 실수 값 샘플링</td>
      <td>hp.uniform(label, low, high)</td>
      <td><code>hp.uniform('learning_rate', 0.01, 0.1)</code></td>
    </tr>
    <tr>
      <td>hp.quniform</td>
      <td>연속적이지만 일정 간격(q)을 갖는 값 샘플링</td>
      <td>hp.quniform(label, low, high, q)</td>
      <td><code>hp.quniform('num_layers', 1, 5, 1)</code></td>
    </tr>
    <tr>
      <td>hp.loguniform</td>
      <td>로그 스케일로 분포된 실수 값 샘플링</td>
      <td>hp.loguniform(label, low, high)</td>
      <td><code>hp.loguniform('reg_param', -3, 0)</code></td>
    </tr>
    <tr>
      <td>hp.randint</td>
      <td>정수 값 샘플링</td>
      <td>hp.randint(label, upper)</td>
      <td><code>hp.randint('num_trees', 1, 100)</code></td>
    </tr>
    <tr>
      <td>hp.choice</td>
      <td>주어진 리스트 중 임의의 값 샘플링</td>
      <td>hp.choice(label, options)</td>
      <td><code>hp.choice('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
    </tr>
    <tr>
      <td>hp.normal</td>
      <td>정규분포에서 값 샘플링</td>
      <td>hp.normal(label, mean, std)</td>
      <td><code>hp.normal('dropout_rate', 0.3, 0.05)</code></td>
    </tr>
    <tr>
      <td>hp.lognormal</td>
      <td>로그 정규분포에서 값 샘플링</td>
      <td>hp.lognormal(label, mean, std)</td>
      <td><code>hp.lognormal('scale', 0, 1)</code></td>
    </tr>
  </tbody>
</table>

In [8]:
!pip install hyperopt

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 28.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 31.1 MB/s eta 0:00:00


### HyperOpt의 핵심 개념
- Search Space :어떤 파라미터를 어떤 범위에서 탐색할지 지정 (ex: 0.01 ~ 1.0)
- Objective Function	평가할 함수 → 모델을 학습시키고, 결과(score)를 반환
- Trials	시도 횟수 (몇 번 실험할지)
- TPE (Tree-structured Parzen Estimator)	똑똑한 탐색 알고리즘 (기본 알고리즘)

In [12]:
from hyperopt import hp

# 검색 공간
search_space = { #hp.quniform = 연속적이지만, 일정 간격(q)을 갖는 값 샘플링
    'x': hp.quniform('x', -10, 10, 1), # hp.quniform(name, low, high, q) low에서 high까지, 간격 q로 나뉘어진 값을 정수처럼 반환
    'y': hp.quniform('y', -15, 15, 1) # y도 마찬가지, 이중에서 x와 동일하게 랜덤하게 하나 선택
}

# 출력 예시 (실제로 뽑히는 값 형태)
# {'x': 3.0, 'y': -12.0}
# {'x': -7.0, 'y': 14.0}

In [13]:
import hyperopt

# 목적 함수
def objective(search_space): #serch_space 하이퍼파라미터 조합 x,y를 받아서, 이 조합이 얼마나 좋은지 계산. 
    x = search_space['x'] # 예) 'x':3.0, 
    y = search_space['y'] # 예) 'y':1.0

    return { #손실 계산
        'loss': x**2 + 20 * y, # loss = x^2+20 *y -> HyperOpt는 이 loss 값을 최소화하려고 노력함. 즉, 값이 적을수록 좋은 조합이라고 판단. 
        'status': hyperopt.STATUS_OK # HyperOpt에게 → "이 결과는 정상적으로 계산된 거야!"라고 알려주는 역할
    }

In [15]:
# HyperOpt의 전체 최적화 과정을 실행하는 핵심 부분이에요.
# 이제 우리가 만든 목적 함수와 탐색 공간을 가지고,
# 최적의 하이퍼파라미터 값을 자동으로 찾아주는 역할을 합니다.
from hyperopt import fmin, tpe, Trials 
# fmin 하이퍼 파라미터 최적화 시작 함수 (최솟값 찾기)
# tpe: "Tree-structured Parzen Estimator" → 똑똑하게 탐색하는 알고리즘
# Trials: 실험 결과(히스토리)를 저장해주는 객체

# 탐색 과정을 저장하는 객체
trials = Trials() # 지금까지 어떤 값으로 시도했는지, 결과는 어땠는지를 전부 기록하는 용도
# 나중에 trials.results로 실험 로그를 확인할 수 있음.

# fmin() : 목적 함수의 최소값을 찾는 함수
best_val = fmin( # fmin()은 손실 함수(loss)가 가장 작아지는 x, y 값을 찾아주는 함수예요.
    fn=objective,           # 목적함수, 손실 계산 함수
    space=search_space,     # 검색공간,  x,y값의 범위 정의
    algo=tpe.suggest,       # 베이지안 최적화 적용, 탐색 알고리즘(TPE 사용: 똑똑한 탐색)     
    max_evals=500,          # 반복 횟수 (총 500번 실험)
    trials=trials           # 탐색과정 저장 (결과 저장용)
)

best_val 
# {'x': np.float64(0.0), 'y': np.float64(-15.0)} 가 loss가 가장 적었던 x, y 조합.

100%|██████████| 500/500 [00:05<00:00, 87.94trial/s, best loss: -300.0] 


{'x': np.float64(-0.0), 'y': np.float64(-15.0)}

In [19]:
# 탐색과정 -> 목적함수 반환값 (loss와 실행 상태) 저장 / 실험 로그 확인 
trials.results

[{'loss': 101.0, 'status': 'ok'},
 {'loss': -124.0, 'status': 'ok'},
 {'loss': 249.0, 'status': 'ok'},
 {'loss': -76.0, 'status': 'ok'},
 {'loss': -119.0, 'status': 'ok'},
 {'loss': 236.0, 'status': 'ok'},
 {'loss': 45.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -44.0, 'status': 'ok'},
 {'loss': 289.0, 'status': 'ok'},
 {'loss': -136.0, 'status': 'ok'},
 {'loss': 84.0, 'status': 'ok'},
 {'loss': -51.0, 'status': 'ok'},
 {'loss': -239.0, 'status': 'ok'},
 {'loss': 236.0, 'status': 'ok'},
 {'loss': 76.0, 'status': 'ok'},
 {'loss': -175.0, 'status': 'ok'},
 {'loss': 101.0, 'status': 'ok'},
 {'loss': 45.0, 'status': 'ok'},
 {'loss': -176.0, 'status': 'ok'},
 {'loss': -279.0, 'status': 'ok'},
 {'loss': -279.0, 'status': 'ok'},
 {'loss': -296.0, 'status': 'ok'},
 {'loss': -180.0, 'status': 'ok'},
 {'loss': -291.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 144.0, 'status': 'ok'},
 {'loss': 309.0, 'status': 'ok'},
 {'loss': -171.0, 'status': 'ok'},
 {'los

In [20]:
# 탐색과정 -> 하이퍼 파라미터값을 딕셔너리(리스트) 형태로 저장
trials.vals

{'x': [np.float64(9.0),
  np.float64(6.0),
  np.float64(-7.0),
  np.float64(8.0),
  np.float64(-1.0),
  np.float64(4.0),
  np.float64(-5.0),
  np.float64(9.0),
  np.float64(6.0),
  np.float64(-7.0),
  np.float64(-2.0),
  np.float64(-8.0),
  np.float64(-3.0),
  np.float64(-1.0),
  np.float64(-6.0),
  np.float64(-4.0),
  np.float64(5.0),
  np.float64(-1.0),
  np.float64(-5.0),
  np.float64(2.0),
  np.float64(1.0),
  np.float64(1.0),
  np.float64(2.0),
  np.float64(-10.0),
  np.float64(3.0),
  np.float64(4.0),
  np.float64(2.0),
  np.float64(3.0),
  np.float64(7.0),
  np.float64(10.0),
  np.float64(1.0),
  np.float64(5.0),
  np.float64(3.0),
  np.float64(7.0),
  np.float64(-0.0),
  np.float64(-3.0),
  np.float64(-0.0),
  np.float64(-2.0),
  np.float64(-10.0),
  np.float64(9.0),
  np.float64(-0.0),
  np.float64(5.0),
  np.float64(-2.0),
  np.float64(-8.0),
  np.float64(-3.0),
  np.float64(7.0),
  np.float64(-4.0),
  np.float64(4.0),
  np.float64(2.0),
  np.float64(6.0),
  np.float64(-6.0),

In [22]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   -- ------------------------------------- 7.6/150.0 MB 39.0 MB/s eta 0:00:04
   ---- ----------------------------------- 16.5/150.0 MB 40.0 MB/s eta 0:00:04
   ------ --------------------------------- 23.6/150.0 MB 37.3 MB/s eta 0:00:04
   -------- ------------------------------- 32.2/150.0 MB 37.9 MB/s eta 0:00:04
   ---------- ----------------------------- 40.6/150.0 MB 38.6 MB/s eta 0:00:03
   ------------- -------------------------- 49.3/150.0 MB 38.7 MB/s eta 0:00:03
   --------------- ------------------------ 58.5/150.0 MB 39.2 MB/s eta 0:00:03
   ------------------ --------------------- 67.6/150.0 MB 39.9 MB/s eta 0:00:03
   -------------------- ------------------- 77.1/150.0 MB 40.3 MB/s eta 0:00:02
   ----------------------- ---------------- 86.5/150.0 MB 40.9 MB/s eta 0:00:02
   ------------------------- -------------- 94.9/150.0 MB 40.7 MB/s eta 0:00:02
   -------------------------- ------------ 103.3/1

- hyperopt를 활용한 XGBoost 하이퍼 파라미터 튜닝

- XGBoost 란? "여러 개의 약한 모델(결정트리)을 합쳐서 점점 더 좋은 예측을 만들어내는 똑똑한 머신러닝 알고리즘"
- XGBoost	eXtreme Gradient Boosting
- Boosting	성능이 낮은 모델(약한 모델)을 여러 개 모아서 점점 더 좋은 모델로 만드는 방식
- Gradient	모델이 얼마나 틀렸는지(오차)를 바탕으로 개선 방향을 찾는 방법

- 예시로, 케익을 먹는데, 첫 번째 사람이 대충 맛 평가 (70) -> 두 번째 사람이 단것같다고해서 단맛 수정 -> 세 번째 사람의 또 다른 의견으로 조정... 이렇게 점점 평가를 보완하면서 최종점수를 만드는데 이걸 매우 빠르고 정확하게 구현한게 XGBoost

- XGBoost는 성능이 낮은 트리들을 반복적으로 보완해가며높은 정확도를 만들어내는, 빠르고 똑똑한 머신러닝 알고리즘이다! 🌳⚡

- ✅ XGBoost의 특징
- 🎯 정확도 높음	대부분의 캐글 대회 1등들이 자주 쓰는 이유
- ⚡ 빠름	일반적인 boosting보다 훨씬 속도 빠름 (병렬 처리, 캐시 최적화 등)
- 🌳 트리 기반	여러 개의 결정 트리(decision tree)를 만들어서 예측
- 🔥 과적합 방지	규제(regularization) 기능 내장 → 너무 복잡해지는 걸 막아줌
- ✅ 범주형/수치형 둘 다 처리 잘함	숫자, 범주형 데이터 모두 잘 다룸
- 📊 분류/회귀/랭킹 모두 가능	classification, regression, ranking 문제에 다 사용 가능

- 💡 어디에 쓰일까?
- 고객 이탈 예측
- 암 진단 (유방암, 당뇨 등)
- 금융 사기 탐지
- 가격 예측
- 캐글(Kaggle) 데이터 대회 우승 모델 ✨

In [25]:
# 유방암 데이터를 이용해서 XGBoost 분류 모델을 만들고,하이퍼파라미터를 HyperOpt를 이용해서 자동으로 최적화하는 코드 

# 📦 코드 구조는 3단계
# 1️⃣ 탐색할 하이퍼파라미터 공간 정의 (search_space)
# 2️⃣ "이 조합이 얼마나 좋은지" 평가하는 함수 작성 (objective 함수)
# 3️⃣ HyperOpt로 최적값 자동으로 찾기 (fmin + Trials)

from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score

data = load_breast_cancer()
X_train, X_test, y_train, y_test =\
    train_test_split(data.data, data.target, random_state=42)

# 1. 검색 공간
search_space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100), # 트리 개수:  100~500 사이에서 100 간격으로 탐색 
    'max_depth': hp.quniform('max_depth', 3, 10, 1), # 트리 깊이: 3~10 사이의 정수 값 탐색 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2), # 학습률: 0.01 ~ 0.2 사이의 실수 탐색
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1) # 컬럼 샘플링 비율: 전체 특징 중 몇 퍼센트만 쓸지 (0.5 ~ 1.0)
}

# 2. 목적 함수 / 이 함수는 HyperOpt가 시도해볼 하이퍼파라미터 조합 ss를 받아서, 그 조합으로 모델을 학습시키고, 정확도를 평가한 후 "loss" 값을 반환합니다.
def xgb_objective(ss):

    xgb_clf = XGBClassifier( # 실제 모델 생성 부분.
        n_estimators=int(ss['n_estimators']), # quniform()은 실수로 값을 주기 때문에 int()로 변환해줘야함.
        max_depth=int(ss['max_depth']),
        learning_rate=ss['learning_rate'],
        colsample_bytree=ss['colsample_bytree']
    )
    mean_acc = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean() # 3-fold 교차 검증으로 모델의 평균 정확도(accuracy)를 구합니다.
    return {
        'loss': -1 * mean_acc,# HyperOpt는 "작은 값"을 최적이라고 생각함 → 그래서 음수로 바꿈 /정확도가 클수록 좋은데, HyperOpt는 "loss를 최소화"하려고 하므로 정확도에 -1을 곱함
        'status': hyperopt.STATUS_OK
    }


# 3. Trials() + fmin()
trials = Trials() # 모든 실험 기록 
best = fmin(  
    fn=xgb_objective, # 목적 함수
    space=search_space, # 하이퍼파라미터 범위
    algo=tpe.suggest, # TPE 알고리즘 사용 (똑똑한 탐색)
    #max_evals=50, # 총 50번 실험
    max_evals=100, 
    trials=trials # 실험 로그 저장 
)
best

100%|██████████| 100/100 [00:22<00:00,  4.48trial/s, best loss: -0.971830985915493]


{'colsample_bytree': np.float64(0.8012544657693954),
 'learning_rate': np.float64(0.17929552306220722),
 'max_depth': np.float64(6.0),
 'n_estimators': np.float64(500.0)}

---

### Optuna
- HyperOpt 이후로 나온 더 발전된 하이퍼 파라미터 최적화 프레임 워크
- 가장 좋은 하이퍼파라미터 조합을 자동으로 찾아주는 똑똑한 도구 (하이퍼파라미터 튜닝 라이브러리)

- ✅ 그럼 기존 HyperOpt랑 뭐가 달라?
- 비교 항목	   /     HyperOpt	                /              Optuna
- 코드 작성	   /     search_space, objective 따로/	하나의 함수 안에서 간단하게 작성 가능
- 속도	     /      빠름	      /                      더 빠르고 똑똑한 알고리즘 (TPE 개선됨)
- 사용성	  /     좋아요	 /                       더 직관적이고, 심플함
- 시각화	  /     없음 (직접 구현해야 함)	  /        자동 시각화 내장! (optuna.visualization)
- 실험 저장	   /    Trials로 수동 저장	   /          자동 저장 가능 + DB 연동도 쉬움

<table border="1">
    <thead>
        <tr>
            <th>함수명</th>
            <th>설명</th>
            <th>사용 방법</th>
            <th>예시 코드</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>suggest_uniform</td>
            <td>연속적인 실수 값 샘플링</td>
            <td>trial.suggest_uniform(name, low, high)</td>
            <td><code>trial.suggest_uniform('learning_rate', 0.01, 0.1)</code></td>
        </tr>
        <tr>
            <td>suggest_discrete_uniform</td>
            <td>연속적이지만 일정 간격(step)을 갖는 값 샘플링</td>
            <td>trial.suggest_discrete_uniform(name, low, high, step)</td>
            <td><code>trial.suggest_discrete_uniform('num_layers', 1, 5, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_loguniform</td>
            <td>로그 스케일로 분포된 실수 값 샘플링</td>
            <td>trial.suggest_loguniform(name, low, high)</td>
            <td><code>trial.suggest_loguniform('reg_param', 1e-3, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_int</td>
            <td>정수 값 샘플링</td>
            <td>trial.suggest_int(name, low, high, step)</td>
            <td><code>trial.suggest_int('num_trees', 1, 100)</code></td>
        </tr>
        <tr>
            <td>suggest_categorical</td>
            <td>주어진 리스트 중 임의의 값 샘플링</td>
            <td>trial.suggest_categorical(name, choices)</td>
            <td><code>trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
        </tr>
        <tr>
            <td>suggest_float</td>
            <td>연속적인 실수 값 샘플링 (<code>step</code> 사용 가능)</td>
            <td>trial.suggest_float(name, low, high, step=None, log=False)</td>
            <td><code>trial.suggest_float('alpha', 0.1, 1.0, step=0.1)</code></td>
        </tr>
    </tbody>
</table>

In [26]:
!pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 39.3 MB/s eta 0:00:00


In [ ]:
import optuna

# 목적 함수
def objective(trial): # 이 함수는 Optuna가 여러 번 호출하면서, x와 y라는 값을 바꿔보고, 그 결과(loss)를 리턴받는 곳.
    x = trial.suggest_uniform('x', -10, 10) # trial.suggest_uniform(...)은 말 그대로 x와 y를 지정된 범위 내에서 랜덤하게 선택하는 함수
    y = trial.suggest_uniform('y', -15, 15)
    return (x -3) ** 2 + (y + 5) ** 2 # 2차원 함수의 최솟값 문제 -> 최솟값을 자동으로 찾아줄거임 
    # 수학적으로 보면, (3, -5)에서 가장 작은 값이 되는 2차 함수, 대입 해보면 0 

# 스터디 생성
study = optuna.create_study(direction="minimize") # Optuna의 스터디 객체를 생성, direction="minimize"는 → 목적 함수의 결과를 최소화하고 싶다는 뜻 (loss 최소화)

# 최적화 실행
study.optimize(objective, n_trials=500) # objective() 함수를 500번 실행하면서, 매번 x, y를 바꿔보고 → 결과를 비교 → 점점 더 좋은 조합 탐색. 
    
# 결과 확인
print(study.best_value) # 	최적의 x, y 조합일 때의 수식 결과값 (최소값)
print(study.best_params) # 그때 사용된 x, y 값 (거의 3과 -5에 가까울 것)

[I 2025-04-09 17:33:17,619] A new study created in memory with name: no-name-6b5b7ae1-0861-451d-83b6-999dc8aa0353
C:\Users\hyuna\AppData\Local\Temp\ipykernel_18888\3508545321.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10, 10)
C:\Users\hyuna\AppData\Local\Temp\ipykernel_18888\3508545321.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  y = trial.suggest_uniform('y', -15, 15)
[I 2025-04-09 17:33:17,621] Trial 0 finished with value: 2.9785798178529217 and parameters: {'x': 3.514329743007277, 'y': -3.3525641944794966}. Best is trial 0 with value: 2.9785798178529217.
[I 2025-04-09 17:33:17,622] Trial 1 finished with value: 14.137540595346495 and parameters: {'x': 6.09

0.0031118768970528022
{'x': 2.952955379964706, 'y': -5.029978002314825}


In [13]:
# !pip install plotly

In [14]:
import optuna.visualization as vis

# 하이퍼 파라미터 중요도 시각화
vis.plot_param_importances(study).show()

In [ ]:
# 최적화 히스토리 시각화
vis.plot_optimization_history(study).show()

- optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [ ]:
# 1. 목적 함수
# Optuna는 여러 개의 조합을 테스트하면서 최적의 파라미터를 찾는데, 이때 각 조합에서 성능을 평가할 "목적 함수" 가 필요합니다.
# trial은 하나의 시도 (하이퍼파라미터 조합)를 의미합니다.
def xgb_optuna_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100), # n_estimators: 트리의 개수 (100부터 500까지, 100 단위)
        'max_depth': trial.suggest_int('max_depth', 3, 10), # max_depth: 트리의 최대 깊이 (3~10 사이 정수)
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2), # learning_rate: 학습률 (0.01~0.2 사이 실수)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0) # colsample_bytree: 트리를 만들 때 사용하는 피처 비율 (0.5~1.0 사이 실수)
    }
    xgb_clf = XGBClassifier(**params) # 위에서 정해진 하이퍼파라미터를 기반으로 XGBoost 분류기를 만듭니다.
    return cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean() # 3-겹 교차 검증(cross-validation)을 수행해서 평균 정확도를 구한 뒤 반환합니다.
# 이 평균 정확도를 기준으로 Optuna가 "좋은 파라미터인지" 판단합니다.

# 2. study 객체 -> 최적화
study = optuna.create_study(direction='maximize') # 최적화 방향을 "최대한 높은 정확도"로 설정합니다 (maximize).
study.optimize(xgb_optuna_objective, n_trials=50) # xgb_optuna_objective 목적 함수를 기준으로 50번 시도하면서 최적의 하이퍼파라미터 조합을 찾습니다.


# 3. 결과 출력
print(study.best_params) # 가장 성능이 좋았던 하이퍼파라미터 조합을 출력합니다.
print(study.best_value) # 그 조합으로 얻은 최고 정확도를 출력합니다.

[I 2025-03-28 14:22:31,507] A new study created in memory with name: no-name-906917ed-2924-4ec2-8eee-ab7fae42112a
C:\Users\Playdata\AppData\Local\Temp\ipykernel_1908\1028312989.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-03-28 14:22:32,003] Trial 0 finished with value: 0.9624413145539906 and parameters: {'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.021000054051863594, 'colsample_bytree': 0.7944320331334334}. Best is trial 0 with value: 0.9624413145539906.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_1908\1028312989.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-03-28 14:22:32,158] Trial 1 finished with value: 0.9624413145539906 and parameters: {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.1341501845475387, 'colsample_bytree': 0.7354020834233754}. Best is trial 0 with value: 0.96

{'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.1659738245986902, 'colsample_bytree': 0.5046056593621138}
0.971830985915493


# 다시 한번 더 용어 개념 정리
### 파라미터 (Parameter) 란?
- 모델이 학습을 통해 스스로 찾아내는 값
- 예) 선형회귀: y = wx + b , w(가중치) b(편향) 이 둘이 파라미터
- 예) 신경망(딥러닝): 각 층의 가중치(weight) 와 편향(bias), 학습을 통해 업데이트 됨

- XGBoost
- 여러 개의 결정 트리를 만들 때, 각 트리의 분기 기준, 리프 노드의 값 등 → 학습 데이터를 보고 결정됨

- 즉, 모델이 학습하면서 자동으로 조정하는 값들이 파라미터입니다.

### 하이퍼파라미터 (Hyperparameter)란?
- 사람이 직접 설정해줘야 하는 값
- 예시:
- 학습률 (learning_rate)

- 트리 개수 (n_estimators)

- 트리 깊이 (max_depth)

- 배치 크기 (batch_size)

- 드롭아웃 비율 (dropout)

- 정규화 계수 (lambda, alpha)

- 활성화 함수 (relu, tanh 등)

- 🧑‍💻 즉, 모델이 학습하기 전에 우리가 직접 설정해야 하는 값들이 하이퍼파라미터입니다.

- 🎯 왜 하이퍼파라미터 튜닝이 중요할까?
- 잘못된 하이퍼파라미터 설정 → 과적합/과소적합, 학습 불안정, 성능 저하

- 최적의 하이퍼파라미터 설정 → 일반화 성능 극대화

- 그래서 우리가 Optuna, GridSearchCV, RandomSearch 같은 도구로 하이퍼파라미터를 "자동 탐색"하려는 거예요.
- 🎓 결론
- 파라미터는 "모델이 배우는 값"

- 하이퍼파라미터는 "우리가 알려주는 설정값"

- 하이퍼파라미터를 잘 조정해야 모델이 좋은 성능을 낼 수 있음

##### HyperOpt vs Optuna

- HyperOpt
    - 'colsample_bytree': np.float64(0.6931939616646488)
    - 'learning_rate': np.float64(0.1541783542739828)
    - 'max_depth': np.float64(8.0)
    - 'n_estimators': np.float64(400.0)

- Optuna
    - 'n_estimators': 500
    - 'max_depth': 9
    - 'learning_rate': 0.1659738245986902
    - 'colsample_bytree': 0.5046056593621138

In [ ]:
from sklearn.metrics import accuracy_score

xgb_hpopt = XGBClassifier(
    n_estimators=400, # 트리 400, 깊이 8, 학습률 0.15, 피처 70% 사용
    max_depth=8,
    learning_rate=0.15,
    colsample_bytree=0.7
)

xgb_optuna = XGBClassifier(  # 트리 500, 깊이 9, 학습률 0.17, 피처 50% 사용
    n_estimators=500,
    max_depth=9,
    learning_rate=0.17,
    colsample_bytree=0.5
)

xgb_hpopt.fit(X_train, y_train) # 두 모델 모두 학습 데이터 (X_train, y_train)로 학습을 진행함
xgb_optuna.fit(X_train, y_train)

hpopt_pred = xgb_hpopt.predict(X_test) # 학습된 모델들이 테스트 데이터(X_test)에 대해 예측값을 만듦
optuna_pred = xgb_optuna.predict(X_test)

print(f"HyperOpt 최적 파라미터 적용: {accuracy_score(y_test, hpopt_pred)}")
print(f"Optuna 최적 파라미터 적용: {accuracy_score(y_test, optuna_pred)}")

HyperOpt 최적 파라미터 적용: 0.958041958041958
Optuna 최적 파라미터 적용: 0.958041958041958
